<div align="right">
Massimo Nocentini<br>
<br>May 29, 2016: Linearization
<br>May 28, 2016: Binary Forests, Sigmoids
<br>May 27, 2016: Binary and Ternary Trees, Dick paths
<br>May 26, 2016: structure generation
</div>
<br>
<div align="center">
<b>Abstract</b><br>
This document collect some examples about *recursively-defined structures*.
</div>

In [1]:
%matplotlib inline
%run ../python-libs/bits.py
%run ../python-libs/timing.py
%run ../python-libs/graycodes.py
%run ../python-libs/symbols.py

In [2]:
# numbering and coordinates are inverses the one of the other                  
def coordinates(group):
    
    def do(anchor):
        c, r = divmod(anchor, group)
        return r, c # simply flip them

    return do

def numbering(group):
    
    def do(coordinate):
        r, c = coordinate
        return r + group*c

    return do

#_______________________________________________________________

def recursive_structures(shapes, start_cell, locating, handle_multiplicities=False):
    "Returns a generator of structures according `shapes` recursion schemas, starting at anchor `init`."
    
    from collections import defaultdict
    
    coordinates, numbering = locating
            
    def gen(generation):

        yield generation

        next_generation = []
        for anchors, stars in generation:
            while stars: # `stars` is an integer to be used as bitmask: 1 in position k 
                         # means that a ★ is present at cell numbered k
                anchor, stars = low_bit_and_clear(stars)
                r, c = coordinates(anchor)
                for s in shapes:
                    for orientation in s(r, c):
                        augmented_stars = stars
                        multiplicities = 1 # are really necessary?
                        
                        for star in orientation:
                            rs, cs, *offsets = star
                            if offsets:
                                r_offset, c_offset = offsets
                                translating_stars = ones_of(augmented_stars)
                                augmented_stars = 0
                                for ts in translating_stars:
                                    ts_r, ts_c = coordinates(ts)
                                    translated = ts_r + r_offset, ts_c + c_offset
                                    augmented_stars = set_bit(augmented_stars, numbering(translated))
                            
                            star_cell = numbering((rs,cs),)
                            multiplicities += 1 * is_on(augmented_stars, star_cell, return_int=True) * int(handle_multiplicities)
                            augmented_stars = set_bit(augmented_stars, star_cell)
                            
                        struct = set_bit(anchors, anchor), augmented_stars
                        next_generation.extend([struct]*multiplicities)

        yield from gen(next_generation)
                
    init_structure = 0b0, 1 << start_cell
    
    return gen([init_structure])

# Binary trees

In [43]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
  ●
★ o ★  
"""
bintree_shape = lambda r, c: [((r+1, c-1), (r+1, c+1))]

In [44]:
shapes = [bintree_shape]

In [3]:
def pretty(structures):
    
    from collections import defaultdict

    strings = []
    for anchors, stars in structures:
    
        table = defaultdict(lambda: ' ')
    
        for a in ones_of(anchors):
            ca = coor(a)
            table[ca] = table[ca] + '|●' if ca in table else '●'
        
        for s in ones_of(stars):
            cs = coor(s)
            table[cs] = table[cs] + '|★' if cs in table else '★'
        
        s = ''
        for r in range(rows):
            s += ' '.join(table[r, c] for c in range(cols)) + '\n'
        
        strings.append(s)
    
    return  ''.join(strings)

In [46]:
rows, cols = 8, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
bin_trees = recursive_structures(shapes, rows*12, (coor, number))
representations = map(pretty, bin_trees)

First generation contains only the *seed* symbol ★

In [47]:
print(next(representations))

                        ★                        
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [48]:
print(next(representations))

                        ●                        
                      ★   ★                      
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [49]:
print(next(representations))

                        ●                        
                      ●   ★                      
                    ★   ★                        
                                                 
                                                 
                                                 
                                                 
                                                 
                        ●                        
                          ●                      
                        ★   ★                    
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [50]:
print(next(representations))

                        ●                        
                      ●   ★                      
                    ●   ★                        
                  ★   ★                          
                                                 
                                                 
                                                 
                                                 
                        ●                        
                      ●   ★                      
                        ●                        
                      ★   ★                      
                                                 
                                                 
                                                 
                                                 
                        ●                        
                      ●   ●                      
                        ★   ★                    
                                                 


Next generation:

In [11]:
print(next(representations))

                        ●                        
                      ●   ★                      
                    ●   ★                        
                  ●   ★                          
                ★   ★                            
                                                 
                                                 
                                                 
                        ●                        
                      ●   ★                      
                    ●   ★                        
                      ●                          
                    ★   ★                        
                                                 
                                                 
                                                 
                        ●                        
                      ●   ★                      
                    ●   ●                        
                      ★   ★                      


# Ternary trees

In [25]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
  ●
★ ★ ★  
"""
threetree_shape = lambda r, c: [((r+1, c-1), (r+1, c), (r+1, c+1))]

In [26]:
shapes = [threetree_shape]

In [28]:
rows, cols = 8, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
three_trees = recursive_structures(shapes, rows*12, (coor, number))
representations = map(pretty, three_trees)

First generation contains only the *seed* symbol ★

In [29]:
print(next(representations))

                        ★                        
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [30]:
print(next(representations))

                        ●                        
                      ★ ★ ★                      
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [31]:
print(next(representations))

                        ●                        
                      ● ★ ★                      
                    ★ ★ ★                        
                                                 
                                                 
                                                 
                                                 
                                                 
                        ●                        
                        ● ★                      
                      ★ ★ ★                      
                                                 
                                                 
                                                 
                                                 
                                                 
                        ●                        
                          ●                      
                        ★ ★ ★                    
                                                 


Next generation:

In [32]:
print(next(representations))

                        ●                        
                      ● ★ ★                      
                    ● ★ ★                        
                  ★ ★ ★                          
                                                 
                                                 
                                                 
                                                 
                        ●                        
                      ● ★ ★                      
                      ● ★                        
                    ★ ★ ★                        
                                                 
                                                 
                                                 
                                                 
                        ●                        
                      ● ● ★                      
                      ★ ★ ★                      
                                                 


Next generation:

In [33]:
print(next(representations))

                        ●                        
                      ● ★ ★                      
                    ● ★ ★                        
                  ● ★ ★                          
                ★ ★ ★                            
                                                 
                                                 
                                                 
                        ●                        
                      ● ★ ★                      
                    ● ★ ★                        
                    ● ★                          
                  ★ ★ ★                          
                                                 
                                                 
                                                 
                        ●                        
                      ● ★ ★                      
                    ● ● ★                        
                    ★ ★ ★                        


# Binary forests

In [68]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
o ★
● ★

● o
★ ★
"""
forest_shape = lambda r, c: [((r-1, c+1), (r, c+1)),
                             ((r+1, c), (r+1, c+1)),]

In [69]:
shapes = [forest_shape]

In [74]:
rows, cols = 8, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
forests = recursive_structures(shapes, rows*12+4, (coor, number)) # be aware that in this case it grows upwards!
representations = map(pretty, forests)

First generation contains only the *seed* symbol ★

In [75]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
                        ★                        
                                                 
                                                 
                                                 



Next generation:

In [76]:
print(next(representations))

                                                 
                                                 
                                                 
                          ★                      
                        ● ★                      
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                        ●                        
                        ★ ★                      
                                                 
                                                 



Next generation:

In [77]:
print(next(representations))

                                                 
                                                 
                            ★                    
                          ● ★                    
                        ● ★                      
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                          ●                      
                        ● ★ ★                    
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                            ★                    


# Dick paths

In [78]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
  ★
( o ★  
"""
dick_shape = lambda r, c: [((r-1, c+1, 0, 2), (r, c+2))]

In [35]:
shapes = [dick_shape]

In [36]:
def pretty(structures):
    
    from collections import defaultdict

    strings = []
    for anchors, stars in structures:
    
        table = defaultdict(lambda: ' ')
    
        for a in ones_of(anchors):
            ca = coor(a)
            table[ca] = table[ca] + '|(' if ca in table else '('
        
        for s in ones_of(stars):
            cs = coor(s)
            table[cs] = table[cs] + '|★' if cs in table else '★'
        
        s = ''
        for r in range(rows):
            s += ' '.join(table[r, c] for c in range(cols)) + '\n'
        
        strings.append(s)
    
    return  ''.join(strings)

In [37]:
rows, cols = 8, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
dick_paths = recursive_structures(shapes, rows-1, (coor, number))
representations = map(pretty, dick_paths)

First generation contains only the *seed* symbol ★

In [38]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
★                                                



Next generation:

In [39]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
  ★                                              
(   ★                                            



Next generation:

In [40]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
                                                 
    ★                                            
  (   ★                                          
(       ★                                        
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
      ★                                          
(   (   ★                                        



Next generation:

In [41]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
      ★                                          
    (   ★                                        
  (       ★                                      
(           ★                                    
                                                 
                                                 
                                                 
                                                 
                                                 
        ★                                        
  (   (   ★                                      
(           ★                                    
                                                 
                                                 
                                                 
                                                 


Next generation:

In [42]:
print(next(representations))

                                                 
                                                 
                                                 
        ★                                        
      (   ★                                      
    (       ★                                    
  (           ★                                  
(               ★                                
                                                 
                                                 
                                                 
                                                 
          ★                                      
    (   (   ★                                    
  (           ★                                  
(               ★                                
                                                 
                                                 
                                                 
                                                 


# Sigmoids

In [49]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
  ★
● o o
    ★
"""
sigmoid_shape = lambda r, c: [((r-1, c+1,), (r+1, c+2,))] #, (r, c+3,)

In [50]:
shapes = [sigmoid_shape]

In [51]:
rows, cols = 9, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
sigmoids = recursive_structures(shapes, 4, (coor, number))
representations = map(pretty, sigmoids)

First generation contains only the *seed* symbol ★

In [52]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
★                                                
                                                 
                                                 
                                                 
                                                 



Next generation:

In [53]:
print(next(representations))

                                                 
                                                 
                                                 
  ★                                              
●                                                
    ★                                            
                                                 
                                                 
                                                 



Next generation:

In [54]:
print(next(representations))

                                                 
                                                 
    ★                                            
  ●                                              
●     ★                                          
    ★                                            
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
●     ★                                          
    ●                                            
        ★                                        
                                                 
                                                 



Next generation:

In [55]:
print(next(representations))

                                                 
      ★                                          
    ●                                            
  ●     ★                                        
●     ★                                          
    ★                                            
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
  ●                                              
●     ★                                          
    ●                                            
        ★                                        
                                                 
                                                 
                                                 
                                                 


Next generation:

In [56]:
print(next(representations))

                                                 
      ★                                          
    ●                                            
  ●     ★                                        
●     ★                                          
    ●                                            
        ★                                        
                                                 
                                                 
        ★                                        
      ●                                          
    ●     ★                                      
  ●     ★                                        
●     ★                                          
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 


# Linearization

In [5]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

""" 
● ★ ★  
"""
linear_shape = lambda r, c: [((r, c+1, 0, 2), (r, c+2))]

In [6]:
shapes = [linear_shape]

In [7]:
rows, cols = 3, 25
coor, number = coordinates(rows), numbering(rows)

linears = recursive_structures(shapes, 1, (coor, number))
representations = map(pretty, linears)

First generation contains only the *seed* symbol ★

In [8]:
print(next(representations))

                                                 
★                                                
                                                 



Next generation:

In [9]:
print(next(representations))

                                                 
● ★ ★                                            
                                                 



Next generation:

In [10]:
print(next(representations))

                                                 
● ● ★ ★ ★                                        
                                                 
                                                 
●   ● ★ ★                                        
                                                 



Next generation:

In [11]:
print(next(representations))

                                                 
● ● ● ★ ★ ★ ★                                    
                                                 
                                                 
● ●   ● ★ ★ ★                                    
                                                 
                                                 
● ●     ● ★ ★                                    
                                                 
                                                 
●   ● ● ★ ★ ★                                    
                                                 
                                                 
●   ●   ● ★ ★                                    
                                                 



Next generation:

In [12]:
print(next(representations))

                                                 
● ● ● ● ★ ★ ★ ★ ★                                
                                                 
                                                 
● ● ●   ● ★ ★ ★ ★                                
                                                 
                                                 
● ● ●     ● ★ ★ ★                                
                                                 
                                                 
● ● ●       ● ★ ★                                
                                                 
                                                 
● ●   ● ● ★ ★ ★ ★                                
                                                 
                                                 
● ●   ●   ● ★ ★ ★                                
                                                 
                                                 
● ●   ●     ● ★ ★                                


---
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Recursive Structures tutorial</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="massimo.nocentini@unifi.it" property="cc:attributionName" rel="cc:attributionURL">Massimo Nocentini</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/massimo-nocentini/competitive-programming/blob/master/tutorials/recursive-structures.ipynb" rel="dct:source">https://github.com/massimo-nocentini/competitive-programming/blob/master/tutorials/recursive-structures.ipynb</a>.